In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import datetime

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280


[WDM] - Driver [/Users/samanthalane/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache


In [3]:
scraped_data= {}

#Getting News Info

url = "https://mars.nasa.gov/news/"
browser.visit(url)

#allow broswer to open
time.sleep(1)

soup = BeautifulSoup(browser.html)

#Grabbing the Titles and the Paragapghs 
slide = soup.find("li", {"class": "slide"})
newsTitle = slide.find("div", {"class": "content_title"}).text.strip()
newsPara = slide.find("div", {"class": "article_teaser_body"}).text.strip()

#Getting image 
base = "https://www.jpl.nasa.gov"
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)


#allow broswer to open
time.sleep(1)

#find image
browser.find_by_id("full_image").click()
time.sleep(1)

browser.find_link_by_partial_text("more info").click()
time.sleep(1)

soup = BeautifulSoup(browser.html)
image = soup.find("img", {"class": "main_image"})

featuredImage = base + image["src"]

#get Mars Facts 
url = "https://space-facts.com/mars/"
browser.visit(url)
time.sleep(1)

table = pd.read_html(url)
df = table[0]
df.columns = ["Criteria", "Values"]
facts = df.to_html(index=False)

# Hemisphere Data 
base = "https://astrogeology.usgs.gov"
url = f"{base}/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)
time.sleep(1)

soup = BeautifulSoup(browser.html)
imgLinks = soup.find("div", {"class":"results"}).findAll("a", {"class":"itemLink"})
imgLinks

links = []
for link in imgLinks: 
    image = link.find("img")
    if (image):
        links.append(base + link["href"])
        
hemiData = []       
for link in links: 
    browser.visit(link)
    time.sleep(1)
    soup = BeautifulSoup(browser.html)
    hemiUrl = soup.find("ul").find("li").find("a")["href"]
    hemiTitle = soup.find("h2", {"class" : "title"}).text.split(" Enhanced")[0]
    hemiData.append({"title": hemiTitle, "url": hemiUrl})
    


browser.quit()

#add data to dictonary 
scraped_data["newsTitle"] = newsTitle
scraped_data["newsPara"] = newsPara
scraped_data["featuredImage"] = featuredImage
scraped_data["marsFacts"] = facts
scraped_data["hemisphere"]= hemiData
scraped_data["last_updated"] = datetime.datetime.now()

/opt/anaconda3/lib/python3.8/site-packages/splinter/driver/webdriver/__init__.py:490: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  warnings.warn(


In [4]:
scraped_data

{'newsTitle': '7 Things to Know About the NASA Rover About to Land on Mars',
 'newsPara': 'The Mars 2020 Perseverance rover, which has started its approach to the Red Planet, will help answer the next logical question in Mars exploration.',
 'featuredImage': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17449_hires.jpg',
 'marsFacts': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>Criteria</th>\n      <th>Values</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <td>Orbit Period:</td>\n      <td>687 